# AniMate Ver. 1.0.0

This notebook is used to prototype the LLM Agent that will act as the brain of the LLM-based Chatbot. Here I:
1. Define the tools available to the agent
2. Experiment with different prompt templates when constructing the agent

## Agent Construction

The API that the agent will use to find up to date anime related information is the [Jikan API](https://jikan.moe/). It parses the website MyAnimeList.net (MAL), one of the largest anime databases available, to satisfy the needs of the API.

The capabilities I need to provide the agent for core functionality are listed below:
- **Retrieve anime synopsis:** Get the anime's synopsis. *Wikipedia*
- **Retrieve anime ID:** Get the MAL ID of the anime. Scraped from the MAL site
- **Retrieve real-time statistics:** Get the live statistics on an anime. *Jikan API endpoint: getAnimeStatistics*
- **Retrieve recommendations:** Get other anime recommendations based on a given anime. *Jikan API endpoint: getAnimeRecommendations*
- **Retrieve sentiment:** Get sentiment around the anime. *Jikan API endpoint: getAnimeReviews*

### Imports

In [ ]:
%pip install fuzzywuzzy
%pip install langchain
%pip install python-Levenshtein
%pip install openai
%pip install tiktoken
%pip install wikipedia

%pip install python-dotenv
%load_ext dotenv
%dotenv

In [90]:
import json, requests, os, yaml, urllib.parse
from enum import Enum
from typing import Type, Optional
from fuzzywuzzy import fuzz
from pydantic import BaseModel, Field

from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_core.tools import ToolException
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import RequestsWrapper

### Retrieve Anime Synopsis

#### Tool

In [117]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wiki_tool = Tool.from_function(
        func=wikipedia.run,
        name="wikipedia_lookup",
        description="get anime synopsis",
        handle_tool_error=True
    )

wikipedia.run("Attack on Titan")

'Page: Attack on Titan\nSummary: Attack on Titan (Japanese: 進撃の巨人, Hepburn: Shingeki no Kyojin, lit.\u2009\'The Advancing Giant\') is a Japanese manga series written and illustrated by Hajime Isayama. It is set in a world where humanity is forced to live in cities surrounded by three enormous walls that protect them from gigantic man-eating humanoids referred to as Titans; the story follows Eren Yeager, who vows to exterminate the Titans after they bring about the destruction of his hometown and the death of his mother. It was serialized in Kodansha\'s monthly magazine Bessatsu Shōnen Magazine from September 2009 to April 2021, with its chapters collected in 34 tankōbon volumes.\nAn anime television series was produced by Wit Studio (seasons 1–3) and MAPPA (season 4). A 25-episode first season was broadcast from April to September 2013, followed by a 12-episode second season broadcast from April to June 2017. A 22-episode third season was broadcast in two parts, with the first 12 episo

#### Test

In [ ]:
tools = [wiki_tool]
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)
result = agent_executor({"input": "Tell me the plot of Jujutsu Kaisen"})

### Retrieve Anime ID

The agent needs the ability to retrieve the MyAnimeList (MAL) ID of a given anime as this ID must be included in the request for almost every endpoint of the Jikan API. Retrieving the MAL ID of a given anime is trivial as this information can simply be scraped from the internet, so the tool for this will simply enable the LLM to scrape the appropriate page of the site for the id using the name of the anime e.g. *getId(anime_name: string) -> int*.

#### Utils

In [64]:
def search_anime_name(animeName: str) -> dict[str, any]:
    url = 'https://api.jikan.moe/v4/anime?q={name}'.format(name=animeName)
    response = requests.get(url)
    response_json = response.json()
    return response_json['data']

def get_best_match(query: str, search_results: list[dict[str, any]]) -> tuple[str, str]:
    # Sort by levenshtein distance (edit distance) between query and title. The closest
    # match will be the final result
    search_results_sorted = sorted(search_results, key=lambda sr: fuzz.ratio(query, sr['title']))
    best_match = search_results_sorted[len(search_results) - 1]
    best_match_title = best_match['title']
    best_match_id = best_match['mal_id'] 
    return best_match_title, best_match_id

# Test
query = "Naruto Shippuden"
search_results = search_anime_name(query)
best_match = get_best_match(query, search_results)
print(best_match)

('Naruto: Shippuuden', 1735)


#### Tool

In [4]:
class AnimeIDLookupSchema(BaseModel):
    query: str = Field(description="anime name")


class AnimeIDLookupTool(BaseTool):
    name = "anime_id_lookup"
    description = "Useful for when you need to retrieve the id of an anime"
    args_schema: Type[AnimeIDLookupSchema] = AnimeIDLookupSchema

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        try:    
            searchResults = search_anime_name(query)
            if searchResults:
                _, bestMatchId = get_best_match(query, searchResults)
                return bestMatchId
            raise ToolException("Anime not found")
        except Exception as e:
            raise ToolException(e)

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("anime_id_lookup does not support async")

#### Test

Experimentation is carried out using OpenAI function calling, however useful resources for creating agents and learning how to use Llama 2 as an agent locally are listed below: 
- https://www.pinecone.io/learn/llama-2/
- https://philschmid.github.io/easyllm/examples/llama2-agent-example/#basic-example-of-using-a-tool-with-llama-2-70b
- https://medium.com/@gil.fernandes/langchain-chat-with-custom-tools-functions-and-memory-e34daa331aa7

In [5]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

tools = [anime_id_lookup_tool]
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)
result = agent_executor({"input": "Tell me the anime ID of Cowboy Bepop"}) # LLM should return 1
result = agent_executor({"input": "Tell me the anime ID of ..."}) # LLM should throw error

### Remaining functionality

The other aspects of the agent's functionality are non-trivial in that they all require allowing the agent to make API calls to the Jikan API. Langchain supports three main ways of doing this:
1. **OpenAPI** - If the API follows the OpenAPI spec, we can use the OpenAPI toolkit to parse the APIs OpenAPI specification and provide the LLM with the available tools directly. Refer to: https://python.langchain.com/docs/use_cases/apis and https://python.langchain.com/docs/integrations/toolkits/openapi
2. **Non-OpenAPI** - If the API doesn't follow the OpenAPI spec, we can define our own documentation for the API to construct the chain. As far as I know there is no standardised way of doing this, so it would require examining the examples in the Langchain docs and going through trial and error. Refer to: https://python.langchain.com/docs/use_cases/apis
3. **Custom Tools** - We define a custom tool for each endpoint required, and under the hood facilitate fetching the APIs response

Thankfully the JikanAPI provides an OpenAPI spec, so approaches 1 & 3 are viable. I will carry out experimentation below and decide which will be optimal for the final chatbot.

### OpenAPI

In [9]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)
requests_wrapper = RequestsWrapper()

# from urllib.request import urlretrieve
# urlretrieve("https://raw.githubusercontent.com/jikan-me/jikan-rest/master/storage/api-docs/api-docs.json","jikan_openapi.yaml")

with open("jikan_openapi.yaml", encoding="utf-8") as f:
    raw_jikan_api_spec = yaml.load(f, Loader=yaml.Loader)
jikan_api_spec = reduce_openapi_spec(raw_jikan_api_spec)

jikan_agent = planner.create_openapi_agent(jikan_api_spec, requests_wrapper, llm)

# Using simple OpenAPI agent
# NOTE: This agent does not have access to the id lookup tool, so the id must be included in the prompt

user_query = (
    "Tell me the plot of the anime Cowboy Bepop. It has an id of 1"
)
jikan_agent.run(user_query)

# OpenAPI agent + ID lookup tool
# Here the OpenAPI agent is used as a tool alongside the id lookup tool. Notice that the prompt no
# longer needs to include the anime ID. Instead, the main LLM is used as an orchestrator, and is instructed
# to pass the id retrieved using the id lookup tool to the OpenAPI agent.

jikan_tool = Tool.from_function(
        func=jikan_agent.run,
        name="anime_api_agent",
        description="useful for when you need to answer anime related queries. The input to this tool should be in the form '[query] anime id: [id]'",
        handle_tool_error=True
    )

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

tools = [anime_id_lookup_tool, jikan_tool]
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

result = agent_executor({"input": "Tell me the plot of the anime Cowboy Bepop"})



> Entering new AgentExecutor chain...

Invoking: `anime_id_lookup` with `{'query': 'Cowboy Bebop'}`


1
Invoking: `anime_api_agent` with `plot anime id: 1`




> Entering new AgentExecutor chain...
Action: api_planner
Action Input: "plot anime id: 1"
Observation: 1. GET /anime/{id} to retrieve information about the anime with id 1.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: GET /anime/1

> Entering new AgentExecutor chain...
I need to make a GET request to the /anime/1 endpoint to retrieve information about the anime with id 1. I will use the requests_get tool to execute this API call.

Action: requests_get
Action Input:
{
  "url": "https://api.jikan.moe/v4/anime/1",
  "params": {},
  "output_instructions": {}
}

Observation: Title: Cowboy Bebop
Type: TV
Status: Finished Airing
Episodes: 26
Airing: No
Aired: Apr 3, 1998 to Apr 24, 1999
Duration: 24 min per ep
Rating: R - 17+ (violence & profanity)
Score: 8.75
Scored by: 944,288 users
Rank: 45
Pop

### Custom Tools

#### Utils

In [54]:
# class JikanEndpoints(Enum):
#     GENERAL_INFO = "https://api.jikan.moe/v4/anime/{id}"
#     EPISODE_INFO = "https://api.jikan.moe/v4/anime/{id}/episodes"
#     CHARACTER_INFO = "https://api.jikan.moe/v4/anime/{id}/characters"

# class JikanResponseKeys(Enum):
#     TITLE = "TITLE"
#     URL = "URL"
#     TYPE = "TYPE"
#     STATUS = "STATUS"
#     NUM_EPISODES = "NUM_EPISODES"
#     EPISODES = "EPISODES"
#     CHARACTERS = "CHARACTERS"
#     AIRING = "AIRING"
#     AIRED = "AIRED"
#     DURATION = "DURATION"
#     RATING = "RATING"
#     SCORE = "SCORE"
#     SCORED_BY = "SCORED BY"
#     RANK = "RANK"
#     SYNOPSIS = "SYNOPSIS"
#     BACKGROUND = "BACKGROUND"
#     LICENSORS = "LICENSORS"
#     ANIMATION_STUDIOS = "ANIMATION STUDIOS [NOT A PRODUCER]"
#     PRODUCERS = "PRODUCERS"
#     GENRES = "GENRES"

# def jikan_get_request(endpoint: JikanEndpoints, animeId: int) -> dict[str, any]:
#     url = endpoint.value.format(id=animeId)
#     response = requests.get(url)
#     response_json = response.json()
#     data = response_json['data']
#     return data

# def construct_dict(keys: list[str], values: list[str]) -> dict[str, str]:
#     new_dict = dict(zip(keys, values))
#     return new_dict

# def parse_names(objList: list[dict[str, any]]) -> str:
#     namesList = list(map(lambda x: x['name'], objList))
#     names = ", ".join(namesList)
#     return names

# def parse_aired_dates(aired: dict[str, any]) -> str:
#     aired_string = 'from {start} to {end}' \
#                     .format(start=aired['from'], end=aired['to'])
#     return aired_string

# def parse_voice_actors(actors: list[dict[str, any]]) -> str:
#     va_info = list(map(lambda a: "Actor Name: {name}, Actor Language: {language}".format(name=a['person']['name'], language=a['language']), actors))
#     return va_info
    
# def parse_characters(characters: list[dict[str, any]]) -> str:
#     char_info = list(map(lambda c: "Name: {name}, Role: {role}, Voice Actors: {actors}" \
#                     .format(name=c['character']['name'], role=c['role'], actors=parse_voice_actors(c['voice_actors'])), characters))
#     return char_info

# def get_general_info(animeId: int) -> str:
#     endpoint = JikanEndpoints.GENERAL_INFO
#     data = jikan_get_request(endpoint, animeId)
    
#     keys = [
#         JikanResponseKeys.TITLE.value, 
#         JikanResponseKeys.URL.value, 
#         JikanResponseKeys.TYPE.value, 
#         JikanResponseKeys.STATUS.value,
#         JikanResponseKeys.NUM_EPISODES.value, 
#         JikanResponseKeys.AIRED.value, 
#         JikanResponseKeys.AIRING.value,
#         JikanResponseKeys.DURATION.value, 
#         JikanResponseKeys.RATING.value, 
#         JikanResponseKeys.SCORE.value, 
#         JikanResponseKeys.SCORED_BY.value,
#         JikanResponseKeys.RANK.value, 
#         JikanResponseKeys.SYNOPSIS.value, 
#         JikanResponseKeys.BACKGROUND.value, 
#         JikanResponseKeys.LICENSORS.value,
#         JikanResponseKeys.ANIMATION_STUDIOS.value, 
#         JikanResponseKeys.PRODUCERS.value, 
#         JikanResponseKeys.GENRES.value
#         ]
#     values = [
#         data['title'], 
#         data['url'], 
#         data['type'], 
#         data['status'], 
#         data['episodes'],
#         data['airing'],
#         parse_aired_dates(data['aired']),
#         data['duration'],
#         data['rating'],
#         data['score'],
#         data['scored_by'],
#         data['rank'],
#         data['synopsis'],
#         data['background'],
#         parse_names(data['licensors']),
#         parse_names(data['studios']),
#         parse_names(data['producers']),
#         parse_names(data['genres'])
#         ]
    
#     data_parsed = construct_dict(keys, values)
#     data_as_string = str(data_parsed)
#     return data_as_string

# def get_character_info(animeId: int) -> str:
#     url = JikanEndpoints.CHARACTER_INFO
#     data = jikan_get_request(url, animeId)
    
#     keys = [
#         JikanResponseKeys.CHARACTERS.value
#         ]
#     values = [
#         parse_characters(data)
#         ]
    
#     data_parsed = construct_dict(keys, values)
#     data_as_string = str(data_parsed)
#     return data_as_string

# def get_anime_info(animeId: int, option: str) -> str:
#     if (option == "general_info"):
#         return get_general_info(animeId)
#     elif (option == "character_info"):
#         return get_character_info(animeId)

# option = "character_info"
# print(len(get_anime_info(20, option)))

86330


In [122]:
class JikanEndpoints(Enum):
    STATISTICS = "https://api.jikan.moe/v4/anime/{id}/statistics"
    RECOMMENDATIONS = "https://api.jikan.moe/v4/anime/{id}/recommendations"
    REVIEWS = "https://api.jikan.moe/v4/anime/{id}/reviews"
    
def jikan_get_request(endpoint: JikanEndpoints, animeId: int, params: Optional[dict[str, str]]) -> dict[str, any]:
    url = endpoint.value.format(id=animeId)
    
    if params:
        params_formatted = urllib.parse.urlencode(params)
        url_with_params = "{url}?{params}".format(url=url, params=params_formatted)
        url = url_with_params
    
    response = requests.get(url)
    response_json = response.json()
    data = response_json['data']
    return data

def format_scores(scores: list[dict[str, any]]) -> str:
    scores_formatted = list(map(lambda s: "{}/10, Votes for score: {}, Percentage of total: {}\n".format(s['score'], s['votes'], s['percentage']), scores))
    return scores_formatted

def get_anime_statistics(animeId: int) -> str:
    endpoint = JikanEndpoints.STATISTICS
    params = { 'preliminary': 'true' }
    data = jikan_get_request(endpoint, animeId, params)

    
    format_string = "SCORES: {}\nSource: MyAnimeList.net"
    scores = format_scores(data['scores'])
    scores_as_string = " ".join(scores)
    data_parsed = format_string.format(scores_as_string)
    return data_parsed

def get_anime_info(animeId: int, option: str) -> str:
    if (option == "statistics"):
        return get_anime_statistics(animeId)
    # elif (option == "recommendations"):
    #     return get_recommendations(animeId)
    # elif (option == "reviews"):
    #     return get_reviews(animeId)

option = "statistics"
print(get_anime_info(20, option))

SCORES: 1/10, Votes for score: 7790, Percentage of total: 0.4
 2/10, Votes for score: 4705, Percentage of total: 0.2
 3/10, Votes for score: 8122, Percentage of total: 0.4
 4/10, Votes for score: 20739, Percentage of total: 1.1
 5/10, Votes for score: 63736, Percentage of total: 3.3
 6/10, Votes for score: 154293, Percentage of total: 8
 7/10, Votes for score: 423556, Percentage of total: 21.9
 8/10, Votes for score: 532774, Percentage of total: 27.5
 9/10, Votes for score: 366766, Percentage of total: 18.9
 10/10, Votes for score: 354569, Percentage of total: 18.3

Source: MyAnimeList.net


#### Tools

In [123]:
class AnimeLookupInputSchema(BaseModel):
    id: int = Field(description="anime id")


class AnimeStatsTool(BaseTool):
    name = "anime_stats"
    description = "Useful for looking up the statistics on an anime, like the scores people have assigned to it"
    args_schema: Type[AnimeLookupInputSchema] = AnimeLookupInputSchema

    def _run(
        self,
        id: int,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        try:    
            anime_stats = get_anime_statistics(id)
            return anime_stats
        except Exception as e:
            raise ToolException(e)

    async def _arun(
        self,
        id: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("anime_stats does not support async")

#### Test

In [124]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

anime_stats_tool = AnimeStatsTool()
anime_stats_tool.handle_tool_error = True

tools = [anime_id_lookup_tool, anime_stats_tool]
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

result = agent_executor({"input": "Tell me what you know about Naruto Shippuden. Please include any relevant stats on the anime as well"})



> Entering new AgentExecutor chain...

Invoking: `anime_id_lookup` with `{'query': 'Naruto Shippuden'}`


1735
Invoking: `wikipedia_lookup` with `Naruto Shippuden`


Page: Naruto (TV series)
Summary: Naruto is a Japanese anime television series based on Masashi Kishimoto's manga series of the same name. The story follows Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the Hokage, the leader of his village. Just like the manga, the anime series is divided into two separate parts: the first series retains the original manga's title and is set in Naruto's pre-teen years. The second series, a direct sequel titled Naruto: Shippuden, takes place during his teens. Both anime series were animated by Pierrot, produced by Aniplex and licensed in North America by Viz Media.
The first anime series aired on TV Tokyo and ran for 220 episodes from October 2002 to February 2007; an English dub produced by Viz Media aired on Cartoon Network and YTV from Septe

### Conclusion

Using OpenAPI has a major drawback in that it naturally causes the LLM to perform inference more than necessary due to the sheer number of endpoints available. As the LLM performs reasoning to respond to a user query, it calls any endpoint that may be helpful, despite the fact that often only one or two endpoints will return the necessary information. Considering that only a subset of the endpoints is necessary to achieve core functionality, this means the cost of inference is unnecessarily high. Thus, for AniMate, I will use the custom tools approach, and only implement tools for a subset of the endpoints. This will keep the number of tokens required for inference down and reduce the complexity of AniMate's implementation.

## Final Agent



In [ ]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wiki_tool = Tool.from_function(
        func=wikipedia.run,
        name="wikipedia_lookup",
        description="get anime synopsis",
        handle_tool_error=True
    )

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

anime_stats_tool = AnimeStatsTool()
anime_stats_tool.handle_tool_error = True

tools = [wiki_tool, anime_id_lookup_tool, anime_stats_tool]
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

result = agent_executor({"input": "Tell me what you know about Naruto Shippuden. Please include any relevant stats on the anime as well"})